# 显示页码

```python
# 获取排序的方式 # 获取分类商品的信息
# sort=default 按照默认id排序
# sort=price 按照商品价格排序
# sort=hot 按照商品销量排序
sort = request.GET.get('sort')

if sort == 'price':
    skus = GoodsSKU.objects.filter(type=type).order_by('price')
elif sort == 'hot':
    skus = GoodsSKU.objects.filter(type=type).order_by('-sales')
else:
    sort = 'default'
    skus = GoodsSKU.objects.filter(type=type).order_by('-id')
            
# 对数据进行分页
paginator = Paginator(skus, 10)

# 获取第page页的内容
try:
    page = int(page)
except Exception as e:
    page = 1

if page > paginator.num_pages:
    page = 1

# 获取第page页的Page实例对象
skus_page = paginator.page(page)

# todo: 进行页码的控制，页面上最多显示5个页码
# 1.总页数小于5页，页面上显示所有页码
# 2.如果当前页是前3页，显示1-5页
# 3.如果当前页是后3页，显示后5页
# 4.其他情况，显示当前页的前2页，当前页，当前页的后2页
num_pages = paginator.num_pages
if num_pages < 5:
    pages = range(1, num_pages+1)
elif page <= 3:
    pages = range(1, 6)
elif num_pages - page <= 2:
    pages = range(num_pages-4, num_pages+1)
else:
    pages = range(page-2, page+3)
```

# 全文检索

- 数据库like '%abc%'查询，效率低
- 搜索引擎：对表中的字段进行关键词分析，并建立索引
- 全文检索框架-haystack
- 全文搜索引擎 whoosh：纯Python编写
- 全文搜索引擎 Elasticsearch：效率较高，Java编写
- pip install django-haystack
- pip install whoosh

```python
# settings.py

INSTALLED_APPS = (
    ...
    'haystack',
)

# 全文搜索框架的配置
HAYSTACK_CONNECTIONS = {
    'default': {
        #使用whoosh引擎
        'ENGINE': 'haystack.backends.whoosh_backend.WhooshEngine',
        #索引文件路径
        'PATH': os.path.join(BASE_DIR, 'whoosh_index'),
    }
}
#当添加、修改、删除数据时，自动生成索引
HAYSTACK_SIGNAL_PROCESSOR = 'haystack.signals.RealtimeSignalProcessor'
```

---

```python
# 在应用的目录中创建search_indexes.py(文件名必须是这样)添加如下内容

from haystack import indexes
from goods.models import GoodsSKU
#指定对于某个类的某些数据建立索引
# 类名无要求，但一般都是模型类名+Index
class GoodsSKUIndex(indexes.SearchIndex, indexes.Indexable):
    text = indexes.CharField(document=True, use_template=True)
    def get_model(self):
        # 返回模型类
        return GoodsSKU
    def index_queryset(self, using=None):
        # 返回表中的所有数据
        return self.get_model().objects.all()
    
# 多个模型类索引：新建一个类，同上
```

---

```python
# 在templates/search/indexes/应用名称/文件夹中创建模型类名小写_text.txt文件 
# 多个模型类索引：新建一个.txt文件，同上
# 例如templates/search/indexes/goods/goodssku_text.txt

# object.属性名，对相应字段建立索引
{{object.name}}     # 把商品名称纳入索引，能够搜索
{{object.desc}} # 把商品简介纳入索引，能够搜索

# goods是detail的外键
{{object.goods.detail}} # 把商品详情纳入索引，能够搜索
```

##### python manage.py rebuild_index 生成索引文件

# 全文检索使用

##### 1.在settings.py中配置url 

* * url(r'^search' ,include('haystack.urls')),

##### 2.表单搜索时设置表单内容如下。 

* * action为/search
* * method为get
* * input的关键字name为q时，value为搜索内容
* * input的关键字name为models时，value为搜索范围（models=app名字.模型名字）
* * 例如：http://127.0.0.1:8000/search?models=goods.goodssku&q=%E8%8D%89%E8%8E%93 

##### 3.准备模板 

* * 在templates/search目录中创建search.html文件
* * 模板中显示haystack应用中视图函数中传入的参数 
* * 三个参数 
* * * query: 关键字
* * * page: 当前页的page对象，注意里面装的是SearchResult,SearchResult的object属性指向的是模型对象
* * * paginator： 分页对象

##### 4.完成具体的搜索结果页面

* * 拷贝list.html为search.html
* * 修改标题等一些文字
* * 删除新品推荐、菜单栏
* * 只保留遍历商品和页码部分
* * 变量名称修改
* * 更改超链接的url

##### 5.可以修改搜索结果的显示数量 

* * settings.py 
* * HAYSTACK_SEARCH_RESULTS_PER_PAGE = 6 # 设置每页6个结果

# 使用jieba分词模块修改whoosh

- pip install jieba

```python
# 在虚拟环境中找到site-packages/haystack/backends/文件夹
# 创建文件ChineseAnalyzer.py，拷贝如下内容进去，保存

import jieba
from whoosh.analysis import Tokenizer, Token
class ChineseTokenizer(Tokenizer):
    def __call__(self, value, positions=False, chars=False,
                 keeporiginal=False, removestops=True,
                 start_pos=0, start_char=0, mode='', **kwargs):
        t = Token(positions, chars, removestops=removestops, mode=mode, **kwargs)
        seglist = jieba.cut(value, cut_all=True)
        for w in seglist:
            t.original = t.text = w
            t.boost = 1.0
            if positions:
                t.pos = start_pos + value.find(w)
            if chars:
                t.startchar = start_char + value.find(w)
                t.endchar = start_char + value.find(w) + len(w)
            yield t
def ChineseAnalyzer():
    return ChineseTokenizer()
```

---

```python
# 复制目录下的whoosh_backend.py文件，改名为whoosh_cn_backend.py

# 添加相对模块引入 
from .ChineseAnalyzer import ChineseAnalyzer
# 修改代码，使用jieba分词
# 查找
analyzer=StemmingAnalyzer()
# 改为
analyzer=ChineseAnalyzer()
```

---

```python
# 使用jieba引擎 
# 修改settings.py中的全文搜索框架配置

# 查找
whoosh_backend
# 改为
whoosh_cn_backend
```

##### 重新建立索引 
- python manage.py rebuild_index